In [2]:
FIRST_NAME = "Asemaneh" # replace with your first name
LAST_NAME = "Nafe" # replace with your last name
STUDENT_ID = 400105285 # replace with your student id
WANDB_ID = "asemanehnafe" # replace with your wandb username
PROJECT_NAME = f"{FIRST_NAME}-{LAST_NAME}-DQN-EXPLORE-HW"
print(f"Project name: {PROJECT_NAME}")

Project name: Asemaneh-Nafe-DQN-EXPLORE-HW


In [3]:
print(f"Check my results at https://wandb.ai/{WANDB_ID}/{PROJECT_NAME}")

Check my results at https://wandb.ai/asemanehnafe/Asemaneh-Nafe-DQN-EXPLORE-HW


In [4]:
# Set DEBUG to True if you are still implementing the code and debugging
# and don't want to make your wandb dashboard messy.
# set DEBUG to False if you are almost done with the implementation
# and want check performance and compare hyperparameters and models
DEBUG = False

In [5]:
!apt install build-essential python3-dev
!git clone https://github.com/DeepRLCourse/Homework-10.git
%pip install swig
%pip install "Homework-10/BootstrapDQN"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 87 not upgraded.
Cloning into 'Homework-10'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 111 (delta 37), reused 108 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (111/111), 1.14 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (37/37), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.2 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.
Processing ./Homework-10/BootstrapDQN
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━

In [7]:
# DO NOT CHANGE THIS BLOCK
from bootstrapdqn import ReplayBuffer, BaseDQNAgent, get_machine, set_wandb_key_form_secrets, envs
import torch
from torch import nn
import wandb
import random
import gymnasium as gym
import ale_py
import time

gym.register_envs(ale_py)

In [8]:
# DO NOT CHANGE THIS BLOCK
TA = True if WANDB_ID == "alireza9" else False
SAVE_CODE = False if TA else True

In [9]:
print(SAVE_CODE)

True


In [10]:
# DO NOT CHANGE THIS BLOCK
# IF YOU CHANGE ANYTHING ABOUT ENVIRONMENTS AND THEIR RUN CONFIGS, YOUR CODE WILL NOT BE GRADED
from pprint import pprint
ENVS = envs()
pprint(ENVS)

{'CartPole': {'env': {'env_config': {}, 'env_name': 'CartPole-v1', 'seed': 43},
              'run': {'max_episodes': 1000,
                      'max_steps': 50000,
                      'max_steps_per_episode': 100000,
                      'max_time': 720.0}},
 'FrozenLake': {'env': {'env_config': {'p': 0.87, 'size': 14},
                        'env_name': 'FrozenLake-v1',
                        'seed': 42},
                'run': {'max_episodes': 1000000,
                        'max_steps': 1000000,
                        'max_steps_per_episode': 100000,
                        'max_time': 14400}},
 'LunarLander': {'env': {'env_config': {},
                         'env_name': 'LunarLander-v3',
                         'seed': 43},
                 'run': {'max_episodes': 100000,
                         'max_steps': 200000,
                         'max_steps_per_episode': 100000,
                         'max_time': 7200}},
 'MountainCar': {'env': {'env_config': {},
         

In [11]:
if not DEBUG:
    set_wandb_key_form_secrets()

your machine is detected as Kaggle


In [12]:
class EpsGreedyDQNAgent(BaseDQNAgent):
    """
    Epsilon-greedy DQN agent.
    """

    def __init__(self, epsilon: float = 0.1, eps_decay: float = 0.999, eps_min: float = 0.01, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.eps_decay = eps_decay
        self.eps_min = eps_min

    def _decay_eps(self):
        """
        Decay the epsilon value.
        """
        self.epsilon = max(self.epsilon * self.eps_decay, self.eps_min)

    def _create_replay_buffer(self, max_size=1000000):
        self.replay_buffer = ReplayBuffer(
            [
                ("state", (self.env.observation_space.shape[0],), torch.float32),
                ("action", (), torch.int64),
                ("reward", (), torch.float32),
                ("next_state", (self.env.observation_space.shape[0],), torch.float32),
                ("done", (), torch.float32),
            ],
            max_size=max_size,
            device=self.device,
        )

    def _create_network(self):
        self.q_network = nn.Sequential(
            nn.Linear(self.env.observation_space.shape[0], 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, self.env.action_space.n),
        ).to(self.device)
        self.q_network.apply(
            lambda m: torch.nn.init.orthogonal_(m.weight, gain=torch.nn.init.calculate_gain("relu"))
            if isinstance(m, nn.Linear)
            else None
        )
        self.target_network = nn.Sequential(
            nn.Linear(self.env.observation_space.shape[0], 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, self.env.action_space.n),
        ).to(self.device)

    def _compute_loss(self, batch):
        """
        Compute the loss for the DQN agent.
        """
        states = batch["state"]
        actions = batch["action"]
        rewards = batch["reward"]
        next_states = batch["next_state"]
        dones = batch["done"]

        q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze()
        next_q_values = self.target_network(next_states).max(1)[0]
        expected_q_values = rewards + (1 - dones) * self.gamma * next_q_values

        loss = nn.SmoothL1Loss()(q_values, expected_q_values)
        return loss

    def _act_in_training(self, state):
        """
        Select an action during training.
        """
        self._decay_eps()
        if torch.rand(1).item() < self.epsilon:
            return self.env.action_space.sample()
        else:
            with torch.no_grad():
                q_values = self.q_network(torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device))
                return q_values.argmax().item()

    def _act_in_eval(self, state):
        """
        Select an action during evaluation.
        """
        with torch.no_grad():
            q_values = self.q_network(torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device))
            return q_values.argmax().item()

    def _wandb_train_step_dict(self):
        log_dict = super()._wandb_train_step_dict()
        log_dict["train_step/epsilon"] = self.epsilon
        return log_dict

    def _save_dict(self):
        save_dict = super()._save_dict()
        save_dict["epsilon"] = self.epsilon
        save_dict["eps_decay"] = self.eps_decay
        save_dict["eps_min"] = self.eps_min
        return save_dict


In [13]:
class MultiHeadQNet(nn.Module):
    def __init__(self, input_dim, output_dim, k):
        super().__init__()
        self.k = k
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )
        self.heads = nn.ModuleList([
            nn.Linear(256, output_dim) for _ in range(k)
        ])

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.orthogonal_(m.weight, gain=torch.nn.init.calculate_gain("relu"))

    def forward(self, x):
        shared_out = self.shared(x)
        return torch.stack([head(shared_out) for head in self.heads], dim=1)  # [B, K, A]


In [20]:
class BootstrapDQNAgent(EpsGreedyDQNAgent):
    def __init__(self, k: int = 10, bernoulli_p: float = 0.5, **kwargs):
        self.k = k
        self.bernoulli_p = bernoulli_p
        self.current_head = 0
        super().__init__(**kwargs)

    def _create_network(self):
        obs_dim = self.env.observation_space.shape[0]
        action_dim = self.env.action_space.n

        self.q_network = MultiHeadQNet(obs_dim, action_dim, self.k).to(self.device)
        self.target_network = MultiHeadQNet(obs_dim, action_dim, self.k).to(self.device)

    def _create_replay_buffer(self, max_size=1000000):
        self.replay_buffer = ReplayBuffer(
            [
                ("state", (self.env.observation_space.shape[0],), torch.float32),
                ("action", (), torch.int64),
                ("reward", (), torch.float32),
                ("next_state", (self.env.observation_space.shape[0],), torch.float32),
                ("done", (), torch.float32),
                ("mask", (self.k,), torch.bool),  # additional field for head masking
            ],
            max_size=max_size,
            device=self.device,
        )

    def _preprocess_add(self, state, action, reward, next_state, done):
        state = torch.as_tensor(state, dtype=torch.float32, device=self.device)
        action = torch.as_tensor(action, dtype=torch.int64, device=self.device)
        reward = torch.as_tensor(reward, dtype=torch.float32, device=self.device)
        next_state = torch.as_tensor(next_state, dtype=torch.float32, device=self.device)
        done = torch.as_tensor(done, dtype=torch.float32, device=self.device)
    
        mask = torch.bernoulli(torch.full((self.k,), self.bernoulli_p, dtype=torch.float32, device=self.device)).bool()
    
        return {
            "state": state,
            "action": action,
            "reward": reward,
            "next_state": next_state,
            "done": done,
            "mask": mask,
        }


    def _compute_loss(self, batch):
        states = batch["state"]
        actions = batch["action"]
        rewards = batch["reward"]
        next_states = batch["next_state"]
        dones = batch["done"]
        masks = batch["mask"]  # [B, K]

        all_q = self.q_network(states)          # [B, K, A]
        all_q_target = self.target_network(next_states)  # [B, K, A]

        loss = 0
        for head in range(self.k):
            head_mask = masks[:, head]
            if head_mask.sum() == 0:
                continue  # skip if no samples for this head

            q = all_q[head_mask, head, :].gather(1, actions[head_mask].unsqueeze(1)).squeeze()
            next_q = all_q_target[head_mask, head, :].max(1)[0]
            target = rewards[head_mask] + (1 - dones[head_mask]) * self.gamma * next_q
            loss += nn.SmoothL1Loss()(q, target)

        return loss / self.k

    def _episode(self):
        super()._episode()
        self.current_head = random.randint(0, self.k - 1)

    def _act_in_training(self, state):
        # self._decay_eps()
        # if torch.rand(1).item() < self.epsilon:
        #     return self.env.action_space.sample()
        # else:
        with torch.no_grad():
            state_tensor = torch.as_tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)
            q_values = self.q_network(state_tensor)[0, self.current_head]
            return q_values.argmax().item()

    def _act_in_eval(self, state):
        with torch.no_grad():
            state_tensor = torch.as_tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)
            q_values = self.q_network(state_tensor)[0].mean(dim=0)  # average over heads
            return q_values.argmax().item()

    def _wandb_train_episode_dict(self):
        log_dict = super()._wandb_train_episode_dict()
        log_dict["train_episode/current_head"] = self.current_head
        return log_dict

    def _save_dict(self):
        save_dict = super()._save_dict()
        save_dict["k"] = self.k
        save_dict["bernoulli_p"] = self.bernoulli_p
        return save_dict
    def train(
        self,
        max_episodes=10000,
        max_steps_per_episode=5000,
        max_steps=1000000,
        max_time=4 * 60 * 60,
        learn_every=10,
        eval_every=10000,
    ):
        """
        Train the agent over multiple episodes, performing learning updates and periodic evaluations.
        """
        start_time = time.time()
        pre_evaluation_step = self._training_step
        max_steps += self._training_step
        finished = False

        for episode in range(max_episodes):
            if episode % 50 == 0:
                print(f"episode: {episode}")
            self.train_mode()
            self.env.action_space.seed(random.randint(0, 1e32 - 1))
            state, _ = self.env.reset(seed=random.randint(0, 1e32 - 1))
            for step in range(max_steps_per_episode):
                action = self.act(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated

                self._step(reward)
                self.add_experience(state, action, reward, next_state, done)

                state = next_state

                if self._total_steps % learn_every == 0:
                    self.learn()

                if done:
                    break

                if self._training_step >= max_steps or time.time() - start_time >= max_time:
                    finished = True
                    break

            self._episode()

            if self._training_step - pre_evaluation_step >= eval_every:
                self.evaluate()
                pre_evaluation_step = self._training_step

            if finished:
                self.evaluate()
                print(f"Trained for {self._training_step} steps.")
                break


In [21]:
class PriorMultiHeadQNet(nn.Module):
    def __init__(self, input_dim, output_dim, k):
        super().__init__()
        self.k = k
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
        )
        self.heads = nn.ModuleList([
            nn.Linear(64, output_dim) for _ in range(k)
        ])
        self._freeze_weights()

    def _freeze_weights(self):
        for param in self.parameters():
            param.requires_grad = False  # Ensure prior is fixed

    def forward(self, x):
        shared_out = self.shared(x)
        return torch.stack([head(shared_out) for head in self.heads], dim=1)  # [B, K, A]


In [22]:
class RPFBootstrapDQNAgent(BootstrapDQNAgent):
    def _create_network(self):
        super()._create_network()

        input_dim = self.env.observation_space.shape[0]
        output_dim = self.env.action_space.n

        self.prior_network = PriorMultiHeadQNet(input_dim, output_dim, self.k).to(self.device)

    def _act_in_training(self, state):
        # self._decay_eps()
        # if torch.rand(1).item() < self.epsilon:
        #     return self.env.action_space.sample()
        # else:
        with torch.no_grad():
            state_tensor = torch.as_tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)
            q = self.q_network(state_tensor)[0, self.current_head] + \
                self.prior_network(state_tensor)[0, self.current_head]
            return q.argmax().item()

    def _act_in_eval(self, state):
        with torch.no_grad():
            state_tensor = torch.as_tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)
            q = self.q_network(state_tensor)
            return q.mean(dim=1).squeeze().argmax().item()

    def _compute_loss(self, batch):
        states = batch["state"]
        actions = batch["action"]
        rewards = batch["reward"]
        next_states = batch["next_state"]
        dones = batch["done"]
        masks = batch["mask"]

        all_q = self.q_network(states) + self.prior_network(states)              # [B, K, A]
        all_q_target = self.target_network(next_states) + self.prior_network(next_states)

        loss = 0
        for head in range(self.k):
            head_mask = masks[:, head]
            if head_mask.sum() == 0:
                continue

            q = all_q[head_mask, head, :].gather(1, actions[head_mask].unsqueeze(1)).squeeze()
            next_q = all_q_target[head_mask, head, :].max(1)[0]
            target = rewards[head_mask] + (1 - dones[head_mask]) * self.gamma * next_q
            loss += nn.SmoothL1Loss()(q, target)

        return loss / self.k


In [23]:
class UEBootstrapDQNAgent(RPFBootstrapDQNAgent):
    def __init__(self, min_ebs: float = 32, xi= None , **kwargs):
        super().__init__(**kwargs)
        self.min_ebs = min_ebs
        self.xi = None  # Will be set dynamically

    def _compute_loss(self, batch):
        states = batch["state"]
        actions = batch["action"]
        rewards = batch["reward"]
        next_states = batch["next_state"]
        dones = batch["done"]
        masks = batch["mask"]  # [B, K]        losses = []
        sigmas = []

        for j in range(self.k):  # Each head
            head_mask = masks[:, j]

            if head_mask.sum() == 0:
                continue  # skip if no samples for this head

            # Q and Prior
            q_values = self.q_network(states)[0, self.current_head]
            prior = self.prior_network(states)[0, self.current_head]
            q_plus_prior = q_values + prior

            with torch.no_grad():
                # Next action from target + prior
                next_q_values = self.target_network(next_states)
                next_prior = self.prior_network(next_states)
                next_q_plus_prior = next_q_values + next_prior
                next_actions = next_q_plus_prior.argmax(dim=1, keepdim=True)

                # Targets across all heads for uncertainty estimation
                targets_all = []
                for l in range(self.k):
                    tq = self.target_network(next_states)
                    tp = self.prior_network(next_states)
                    target = tq[0, self.current_head] + tp[0, self.current_head]
                    targets_all.append(target)
                targets_all = torch.stack(targets_all, dim=0)  # [k, B, 1]

                # Variance-based uncertainty
                sigma2 = targets_all.var(dim=0, unbiased=False).squeeze(-1)  # [B]
                sigmas.append(sigma2)

                # Compute optimal ξ for EBS
                xi = self._solve_for_xi(sigma2, self.min_ebs)
                self.xi = xi

                weights = 1 / (self.gamma**2 * sigma2 + xi)
                targets = rewards + self.gamma * next_q_plus_prior.gather(1, next_actions) * (1 - dones)
                td_error = q_plus_prior - targets

                w = weights.unsqueeze(1)
                loss = (w * td_error**2).sum() / w.sum()
                losses.append(loss)

        return sum(losses)

    def _solve_for_xi(self, sigma2, min_ebs, tol=1e-4, max_iter=50):
        lower, upper = 1e-6, 10.0
        for _ in range(max_iter):
            xi = (lower + upper) / 2
            weights = 1 / (self.gamma**2 * sigma2 + xi)
            num = weights.sum() ** 2
            denom = (weights ** 2).sum()
            ebs = num / denom
            if abs(ebs - min_ebs) < tol:
                return xi
            if ebs < min_ebs:
                lower = xi
            else:
                upper = xi
        return xi  # fallback

    def _wandb_train_step_dict(self):
        log_dict = super()._wandb_train_step_dict()
        log_dict["xi"] = self.xi
        return log_dict

    def _save_dict(self):
        save_dict = super()._save_dict()
        save_dict["min_ebs"] = self.min_ebs
        return save_dict

In [24]:
# env = ["FrozenLake", "CartPole", "MountainCar", "SeaQuest", "LunarLander"][3]
env = "MountainCar"
print(f"{env} is selected.")

base_agent_config = {
    **ENVS[env]["env"],
    "default_batch_size": 128,
    "gamma": 0.99,
    "learning_rate": 3e-4,
    "replay_buffer_capacity":100_000,
    "tau": 5e-3,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "gradient_norm_clip": 10.0,
    "start_training_after": 1000,
    "normalize_rewards": False,
    "scale_rewards": None
}

base_run_config = {
    **ENVS[env]["run"],
    "learn_every": 1,  # Apply learning every n steps of rollout
    "eval_every": 10_000,  # Evaluate model approximately every n steps
}

MountainCar is selected.


In [25]:
eps_greedy_config = {
    **base_agent_config,
    "eps_decay": 0.9999,
    "eps_min": 0.01,
    "epsilon": 1.0,
}

In [26]:
bootstrap_dqn_config = {
    **eps_greedy_config,
    "k": 5,
    "bernoulli_p": 1,
}

In [27]:
rpf_bootstrap_dqn_config = {
    **bootstrap_dqn_config,
}

In [28]:
ue_bootstrap_dqn_config = {
    **rpf_bootstrap_dqn_config,
    "xi": 0.1,
}

### Epsilon Greedy DQN

In [ ]:
# DON'T CHANGE THIS BLOCK
try:
    eps_greedy_dqn_agent.train(**base_run_config)
    wandb_run.finish()
except KeyboardInterrupt:
    pass

In [ ]:
# DON'T CHANGE THIS BLOCK
wandb_config = {
    "project": PROJECT_NAME,
    "name": "eps_greedy",
    "config": {**eps_greedy_config, **base_run_config, "machine": get_machine()},
    "save_code": SAVE_CODE,
    "tags": ["dqn", "eps_greedy"],
}

if DEBUG:
    wandb_run = None
else:
    wandb_run = wandb.init(**wandb_config)

eps_greedy_dqn_agent = EpsGreedyDQNAgent(wandb_run=wandb_run, **eps_greedy_config)

### Bootstrap DQN

In [ ]:
# DON'T CHANGE THIS BLOCK
wandb_config = {
    "project": PROJECT_NAME,
    "name": "bootstrap",
    "save_code": SAVE_CODE,
    "tags": ["dqn", "bootstrap"],
}

wandb_config["config"] = {} if TA else {**bootstrap_dqn_config, **base_run_config, "machine": get_machine()}

if DEBUG:
    wandb_run = None
else:
    wandb_run = wandb.init(**wandb_config)



In [ ]:
bootstrap_dqn_agent = BootstrapDQNAgent(wandb_run=wandb_run, **bootstrap_dqn_config)

In [ ]:
# DON'T CHANGE THIS BLOCK
try:
    bootstrap_dqn_agent.train(**base_run_config)
    wandb_run.finish()
except KeyboardInterrupt:
    pass

### Bootstrap DQN with Randomized Prior Function

In [21]:
# DON'T CHANGE THIS BLOCK
wandb_config = {
    "project": PROJECT_NAME,
    "name": "randomized_prior",
    "save_code": SAVE_CODE,
    "tags": ["dqn", "rpf_bootstrap"],
}

wandb_config["config"] = {} if TA else {**rpf_bootstrap_dqn_config, **base_run_config, "machine": get_machine()}

if DEBUG:
    wandb_run = None
else:
    wandb_run = wandb.init(**wandb_config)

rpf_bootstrap_dqn_agent = RPFBootstrapDQNAgent(wandb_run=wandb_run, **rpf_bootstrap_dqn_config)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: asemanehnafe to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [22]:
print(wandb_config)

{'project': 'Asemaneh-Nafe-DQN-EXPLORE-HW', 'name': 'randomized_prior', 'save_code': True, 'tags': ['dqn', 'rpf_bootstrap'], 'config': {'env_name': 'MountainCar-v0', 'env_config': {}, 'seed': 43, 'default_batch_size': 128, 'gamma': 0.99, 'learning_rate': 0.0003, 'replay_buffer_capacity': 100000, 'tau': 0.005, 'device': 'cpu', 'gradient_norm_clip': 10.0, 'start_training_after': 1000, 'normalize_rewards': False, 'scale_rewards': None, 'eps_decay': 0.9999, 'eps_min': 0.01, 'epsilon': 1.0, 'k': 5, 'bernoulli_p': 1, 'max_episodes': 100000, 'max_steps': 300000, 'max_steps_per_episode': 100000, 'max_time': 9000.0, 'learn_every': 1, 'eval_every': 10000, 'machine': 'Kaggle'}}


In [23]:
# DON'T CHANGE THIS BLOCK
try:
    rpf_bootstrap_dqn_agent.train(**base_run_config)
    wandb_run.finish()
except KeyboardInterrupt:
    pass

episode: 0


/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 50


error: XDG_RUNTIME_DIR not set in the environment.
/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 100


...

/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 1750
episode: 1800


/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


Trained for 300000 steps.


eval_episode/episode_length,██▇██▆█▇▆▆▆▇▆█▆██▁█████▂▅▁▅▁██
eval_episode/mean_reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_episode/sum_reward,▁▁▂▁▁▃▁▂▃▃▃▂▃▁▃▁▁█▁▁▁▁▁▇▄█▄█▁▁
train_episode/current_head,▃█▅▅▅▅▅▅▆▆▅█▅▁▆▆▅█▃▆▆▅▃▆▃▆▅█▅▅▆▅▅█▆▃▆▅▁▅
train_episode/episode_length,██████▆██▆██▆▇▇▆▇▇█▆▇▁▁█▇██▃▁▂████▂▆▁▅▅█
train_episode/mean_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▂▃▃▄▅▆▇▇█
train_episode/mean_return,▁▁▁▂▂▄▄▅▅▄▄▄▄▄▄▄▄▄▄▅▇▇▇▇▇▇▆▆▇▇▇▇▇███████
train_episode/mean_reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_episode/sum_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▅▆▄█▆▅
train_episode/sum_reward,▁▁▁▁▁▁▁▆▁▁▁▂▁▁▁▃▃▃▂▂▂▅▆█▂▂▃▄▁▁█▁▆▁██▃▄▂▁
train_episode/var_return,▁▁▁▄▅▄▄▃▃▃▃▂▂▃▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████


### Uncertainty Estimation for Sample Efficient RPF Bootstrap DQN

# bounes 1

In [29]:
# DON'T CHANGE THIS BLOCK
wandb_config = {
    "project": PROJECT_NAME,
    "name": "uncertainty_estimation",
    "save_code": SAVE_CODE,
    "tags": ["dqn", "ue_bootstrap"],
}

wandb_config["config"] = {} if TA else {**ue_bootstrap_dqn_config, **base_run_config, "machine": get_machine()}

if DEBUG:
    wandb_run = None
else:
    wandb_run = wandb.init(**wandb_config)

print(wandb_config)



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: asemanehnafe to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'project': 'Asemaneh-Nafe-DQN-EXPLORE-HW', 'name': 'uncertainty_estimation', 'save_code': True, 'tags': ['dqn', 'ue_bootstrap'], 'config': {'env_name': 'MountainCar-v0', 'env_config': {}, 'seed': 43, 'default_batch_size': 128, 'gamma': 0.99, 'learning_rate': 0.0003, 'replay_buffer_capacity': 100000, 'tau': 0.005, 'device': 'cpu', 'gradient_norm_clip': 10.0, 'start_training_after': 1000, 'normalize_rewards': False, 'scale_rewards': None, 'eps_decay': 0.9999, 'eps_min': 0.01, 'epsilon': 1.0, 'k': 5, 'bernoulli_p': 1, 'xi': 0.1, 'max_episodes': 100000, 'max_steps': 300000, 'max_steps_per_episode': 100000, 'max_time': 9000.0, 'learn_every': 1, 'eval_every': 10000, 'machine': 'Kaggle'}}


In [37]:
class UEBootstrapDQNAgent(RPFBootstrapDQNAgent):
    def __init__(self, min_ebs: float = 32, xi = None, **kwargs):
        super().__init__(**kwargs)
        self.min_ebs = min_ebs  # Target effective batch size
        self.xi = None

    def _compute_loss(self, batch):
        states = batch["state"]            # [B, S]
        actions = batch["action"].unsqueeze(1)  # [B,1]
        rewards = batch["reward"].unsqueeze(1)  # [B,1]
        next_states = batch["next_state"]
        dones = batch["done"].unsqueeze(1)      # [B,1]
        masks = batch["mask"]             # [B, K]

        B = states.size(0)

        all_q = self.q_network(states) + self.prior_network(states)          # [B, K, A]
        all_q_tgt = self.target_network(next_states) + self.prior_network(next_states)  # [B, K, A]

        total_loss = 0.0
        log_ebs = []

        for j in range(self.k):
            mask_j = masks[:, j]           # [B]
            if mask_j.sum() == 0:
                continue

            idx = mask_j.nonzero(as_tuple=True)[0]
            s_j = states[idx]
            a_j = actions[idx]
            r_j = rewards[idx]
            ns_j = next_states[idx]
            d_j = dones[idx]

            q_j = (all_q[idx, j]          # [n_j, A]
                   .gather(1, a_j))        # [n_j,1]

            with torch.no_grad():
                tgt_j = all_q_tgt[idx, j]
                a_prime = tgt_j.argmax(dim=1, keepdim=True)  # [n_j,1]
                T_j = r_j + self.gamma * tgt_j.gather(1, a_prime) * (1 - d_j)  # [n_j,1]

                # Compute uncertainty across heads
                x_l = []
                for l in range(self.k):
                    tmp = all_q_tgt[idx, l].gather(1, a_prime)
                    x_l.append(tmp)
                X = torch.stack([tmp.squeeze(1) for tmp in x_l], dim=1)  # [n_j, k]
                sigma2 = X.var(dim=1, unbiased=False, keepdim=True)      # [n_j, 1]


            # Solve for xi such that EBS ≥ min_ebs
            xi = self._solve_for_xi(sigma2.squeeze(), self.min_ebs)
            self.xi = xi

            weights = 1.0 / (self.gamma ** 2 * sigma2 + xi)  # [n_j,1]

            loss_j = (weights * (q_j - T_j) ** 2).sum() / weights.sum()
            total_loss += loss_j
            log_ebs.append(((weights.sum() ** 2) / (weights ** 2).sum()).item())

        loss = total_loss / self.k
        self._last_ebs = torch.tensor(log_ebs).mean().item()
        return loss

    def _solve_for_xi(self, sigma2, min_ebs, tol=1e-3, max_iter=50):
        lo, hi = 1e-8, 1e3
        for _ in range(max_iter):
            xi = (lo + hi) / 2
            w = 1.0 / (self.gamma ** 2 * sigma2 + xi)
            ebs = (w.sum() ** 2) / (w ** 2).sum()
            if abs(ebs - min_ebs) < tol:
                return xi
            if ebs < min_ebs:
                lo = xi
            else:
                hi = xi
        return xi

    def _wandb_train_step_dict(self):
        log = super()._wandb_train_step_dict()
        log["train_step/xi"] = self.xi
        log["train_step/EBS"] = self._last_ebs
        return log

    def _save_dict(self):
        save = super()._save_dict()
        save["min_ebs"] = self.min_ebs
        return save


In [38]:
ue_bootstrap_dqn_agent = UEBootstrapDQNAgent(wandb_run=wandb_run, **ue_bootstrap_dqn_config)

In [39]:
# DON'T CHANGE THIS BLOCK
ue_bootstrap_dqn_agent.train(**base_run_config)
wandb_run.finish()

episode: 0


wandb: WARNING Tried to log to step 200 that is less than the current step 800. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 400 that is less than the current step 800. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 600 that is less than the current step 800. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
error: XDG_RUNTIME_DIR not set in the environment.


episode: 50


/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 100


...

/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 1200


/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 1250
Trained for 229084 steps.


eval_episode/episode_length,█████████████████▁▆█▆▅█
eval_episode/mean_reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_episode/sum_reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▃▁▃▄▁
train_episode/current_head,▁▁▅▁▆▆▆▆▅█▅▁▆▆▆▅██▃▃▁▅▃▃▆▆▁▃▆▁█▅▃▅▆▁▁▁▃█
train_episode/episode_length,████████████████████▂▂▃▂▂█▂▁▆▆▄▄█▂▂▅▇▂▇▆
train_episode/mean_loss,▁▂▃▃▄▅▅▆▆▆▄▄▅▆▆▆▅▇█▄▅▄▄▅▃▃▃▃▃▃▂▃▄▂▂▂▂▂▃▂
train_episode/mean_return,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▅▆▇▇▇█████
train_episode/mean_reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_episode/sum_loss,▁▁▁▃▃▄▅▄▄▃▄▆▄▄▅▄▄▅▄▄██▆▇▄▃▄▃▂▃▂▂▂▂▂▁▂▁▂▁
train_episode/sum_reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▄▄▂▁▄▂▂▃▂▂█
train_episode/var_return,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▄▅▅▆▆▇▇▇█████


# bounes 2 IV-DQN

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from copy import deepcopy


class VarQNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.mu_head = nn.Linear(64, output_dim)
        self.var_head = nn.Linear(64, output_dim)

    def forward(self, x):
        h = self.shared(x)
        mu = self.mu_head(h)
        var = F.softplus(self.var_head(h)) + 1e-4  # ensure positivity
        return mu, var


def ivrl_loss(mu_pred, target, sigma2_pred, xi):
    inv_sigma2 = 1.0 / (sigma2_pred + 1e-6)
    diff = target - mu_pred
    return torch.mean(0.5 * torch.log(sigma2_pred + 1e-6) + 0.5 * xi * diff.pow(2) * inv_sigma2)


def compute_mixture_variance(mus, vars_):
    mean_all = mus.mean(dim=0)
    return vars_.mean(dim=0) + (mus ** 2).mean(dim=0) - mean_all ** 2


def estimate_xi(sigma2, mebs):
    # Use a simple iterative method to estimate xi to meet EBS >= MEBS
    xi = 1.0
    for _ in range(10):
        weights = 1.0 / (sigma2 + 1e-6)
        ebs = (weights.sum() ** 2) / (weights ** 2).sum()
        if ebs >= mebs:
            break
        xi *= 1.5
    return xi


class UEBootstrapDQNAgent:
    def __init__(self, state_dim, action_dim, N=5, gamma=0.99, tau=0.01, lr=1e-3, device='cuda'):
        self.N = N
        self.gamma = gamma
        self.tau = tau
        self.device = device
        self.q_nets = [VarQNetwork(state_dim, action_dim).to(device) for _ in range(N)]
        self.target_nets = [deepcopy(net).to(device) for net in self.q_nets]
        self.prior_nets = [VarQNetwork(state_dim, action_dim).to(device).eval() for _ in range(N)]
        for p in self.prior_nets:
            for param in p.parameters():
                param.requires_grad = False
        self.optimizers = [torch.optim.Adam(net.parameters(), lr=lr) for net in self.q_nets]
        self.replay_buffer = []

    def select_action(self, state):
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
        i = random.randint(0, self.N - 1)
        with torch.no_grad():
            mu, _ = self.q_nets[i](state)
            prior, _ = self.prior_nets[i](state)
            q = mu + prior
        return torch.argmax(q, dim=1).item()

    def store(self, transition):
        self.replay_buffer.append(transition)

    def train_step(self, batch_size=32, delta_rpf=0.0, p_mask=0.5, mebs=8):
        if len(self.replay_buffer) < batch_size:
            return

        minibatch = random.sample(self.replay_buffer, batch_size)
        states, actions, next_states, rewards, dones = zip(*minibatch)
        states = torch.tensor(states, dtype=torch.float32).to(self.device)
        actions = torch.tensor(actions, dtype=torch.long).unsqueeze(1).to(self.device)
        next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float32).unsqueeze(1).to(self.device)

        for j in range(self.N):
            mask = (torch.rand(batch_size) < p_mask).float().to(self.device).unsqueeze(1)

            mu_q, var_q = self.q_nets[j](states)
            mu_q = mu_q.gather(1, actions)
            var_q = var_q.gather(1, actions)

            with torch.no_grad():
                mu_next, _ = self.target_nets[j](next_states)
                prior_next, _ = self.prior_nets[j](next_states)
                next_q = mu_next + delta_rpf * prior_next
                next_actions = torch.argmax(next_q, dim=1, keepdim=True)

                mu_tgt = []
                var_tgt = []
                for l in range(self.N):
                    mu_l, var_l = self.target_nets[l](next_states)
                    prior_l, _ = self.prior_nets[l](next_states)
                    mu = mu_l.gather(1, next_actions) + delta_rpf * prior_l.gather(1, next_actions)
                    var = var_l.gather(1, next_actions)
                    mu_tgt.append(mu)
                    var_tgt.append(var)

                mu_tgt = torch.stack(mu_tgt)
                var_tgt = torch.stack(var_tgt)
                mix_var = compute_mixture_variance(mu_tgt, var_tgt)
                mean_target = mu_tgt[j]
                target = rewards + self.gamma * (1 - dones) * mean_target

            xi = estimate_xi(mix_var, mebs)

            loss = ivrl_loss(mu_q, target, var_q, xi)
            self.optimizers[j].zero_grad()
            loss.backward()
            self.optimizers[j].step()

            # Soft update
            for param, tgt_param in zip(self.q_nets[j].parameters(), self.target_nets[j].parameters()):
                tgt_param.data.copy_(self.tau * param.data + (1.0 - self.tau) * tgt_param.data)

    def save(self, path):
        torch.save([net.state_dict() for net in self.q_nets], path)

    def load(self, path):
        states = torch.load(path)
        for net, state in zip(self.q_nets, states):
            net.load_state_dict(state)


In [60]:
class UEBootstrapDQNAgent(RPFBootstrapDQNAgent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        state_dim = self.env.observation_space.shape[0]
        action_dim = self.env.action_space.n
        device = kwargs.get("device", "cuda")

        self.q_nets = [VarQNetwork(state_dim, action_dim).to(device) for _ in range(self.k)]
        self.target_nets = [deepcopy(net).to(device) for net in self.q_nets]

        self.prior_nets = [VarQNetwork(state_dim, action_dim).to(device).eval() for _ in range(self.k)]
        for p in self.prior_nets:
            for param in p.parameters():
                param.requires_grad = False
        self.optimizers = [torch.optim.Adam(net.parameters(), lr=kwargs['learning_rate']) for net in self.q_nets]

    def train_step(self, batch_size=32, delta_rpf=0.0, p_mask=0.5, mebs=8):
        if len(self.replay_buffer) < batch_size:
            return

        minibatch = random.sample(self.replay_buffer, batch_size)
        states, actions, next_states, rewards, dones = zip(*minibatch)
        states = torch.tensor(states, dtype=torch.float32).to(self.device)
        actions = torch.tensor(actions, dtype=torch.long).unsqueeze(1).to(self.device)
        next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float32).unsqueeze(1).to(self.device)

        for j in range(self.N):
            mask = (torch.rand(batch_size) < p_mask).float().to(self.device).unsqueeze(1)

            mu_q, var_q = self.q_nets[j](states)
            mu_q = mu_q.gather(1, actions)
            var_q = var_q.gather(1, actions)

            with torch.no_grad():
                mu_next, _ = self.target_nets[j](next_states)
                prior_next, _ = self.prior_nets[j](next_states)
                next_q = mu_next + delta_rpf * prior_next
                next_actions = torch.argmax(next_q, dim=1, keepdim=True)

                mu_tgt = []
                var_tgt = []
                for l in range(self.N):
                    mu_l, var_l = self.target_nets[l](next_states)
                    prior_l, _ = self.prior_nets[l](next_states)
                    mu = mu_l.gather(1, next_actions) + delta_rpf * prior_l.gather(1, next_actions)
                    var = var_l.gather(1, next_actions)
                    mu_tgt.append(mu)
                    var_tgt.append(var)

                mu_tgt = torch.stack(mu_tgt)
                var_tgt = torch.stack(var_tgt)
                mix_var = compute_mixture_variance(mu_tgt, var_tgt)
                mean_target = mu_tgt[j]
                target = rewards + self.gamma * (1 - dones) * mean_target

            xi = estimate_xi(mix_var, mebs)

            loss = ivrl_loss(mu_q, target, var_q, xi)
            self.optimizers[j].zero_grad()
            loss.backward()
            self.optimizers[j].step()

            for param, tgt_param in zip(self.q_nets[j].parameters(), self.target_nets[j].parameters()):
                tgt_param.data.copy_(self.tau * param.data + (1.0 - self.tau) * tgt_param.data)


In [43]:
# DON'T CHANGE THIS BLOCK
wandb_config = {
    "project": PROJECT_NAME,
    "name": "uncertainty_estimation",
    "save_code": SAVE_CODE,
    "tags": ["dqn", "ue_bootstrap"],
}

wandb_config["config"] = {} if TA else {**ue_bootstrap_dqn_config, **base_run_config, "machine": get_machine()}

if DEBUG:
    wandb_run = None
else:
    wandb_run = wandb.init(**wandb_config)

print(wandb_config)



{'project': 'Asemaneh-Nafe-DQN-EXPLORE-HW', 'name': 'uncertainty_estimation', 'save_code': True, 'tags': ['dqn', 'ue_bootstrap'], 'config': {'env_name': 'MountainCar-v0', 'env_config': {}, 'seed': 43, 'default_batch_size': 128, 'gamma': 0.99, 'learning_rate': 0.0003, 'replay_buffer_capacity': 100000, 'tau': 0.005, 'device': 'cpu', 'gradient_norm_clip': 10.0, 'start_training_after': 1000, 'normalize_rewards': False, 'scale_rewards': None, 'eps_decay': 0.9999, 'eps_min': 0.01, 'epsilon': 1.0, 'k': 5, 'bernoulli_p': 1, 'xi': 0.1, 'max_episodes': 100000, 'max_steps': 300000, 'max_steps_per_episode': 100000, 'max_time': 9000.0, 'learn_every': 1, 'eval_every': 10000, 'machine': 'Google Colab'}}


In [61]:
ue_bootstrap_dqn_agent = UEBootstrapDQNAgent(wandb_run=wandb_run, **rpf_bootstrap_dqn_config)

In [ ]:
# DON'T CHANGE THIS BLOCK
ue_bootstrap_dqn_agent.train(**base_run_config)
wandb_run.finish()

episode: 0


/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 50


/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 100


...

episode: 1350


/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 1400


/usr/local/lib/python3.11/dist-packages/bootstrapdqn/base_agent.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state, device=self.device),


episode: 1450
episode: 1500


I accidentally closed the Kaggle tab, but the run continued and the results are available on Weights & Biases.